### Import Libraries


In [7]:
import pandas as pd 
import numpy as np

### Import Data

In [2]:
df = pd.read_csv(r'C:\Users\HOLA00067621\OneDrive - Noblex LTD\Desktop\Data analyst test task dataset - Scandiweb.csv')

### EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711189 entries, 0 to 711188
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   event_date             711189 non-null  int64  
 1   event_name             711189 non-null  object 
 2   channel_grouping_user  711189 non-null  object 
 3   device_category        711189 non-null  object 
 4   session_id             711189 non-null  int64  
 5   user_pseudo_id         711189 non-null  float64
 6   item_category          259699 non-null  object 
 7   unique_items           259699 non-null  float64
 8   item_quantity          259699 non-null  float64
 9   revenue                1240 non-null    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 54.3+ MB


In [5]:
df['event_date'] = pd.to_datetime(df['event_date'].astype(str), format='%Y-%m-%d')
df.head(5)

,event_date,event_name,channel_grouping_user,device_category,session_id,user_pseudo_id,item_category,unique_items,item_quantity,revenue
0,2022-10-01,session_start,Paid Search,mobile,1664651196,1.029658e+09,NaN,NaN,NaN,NaN
1,2022-10-01,session_start,Paid Search,mobile,1664656701,1.147645e+09,NaN,NaN,NaN,NaN
2,2022-10-01,session_start,Paid Search,mobile,1664643763,1.483774e+07,NaN,NaN,NaN,NaN
3,2022-10-01,view_search_results,Paid Search,mobile,1664643763,1.483774e+07,NaN,NaN,NaN,NaN
4,2022-10-01,session_start,Paid Search,mobile,1664669276,1.618377e+09,NaN,NaN,NaN,NaN


Let's see some basic statistical information 

In [53]:
df.describe()

,session_id,user_pseudo_id,unique_items,item_quantity,revenue
count,7.111890e+05,7.111890e+05,259699.000000,259699.000000,1.240000e+03
mean,1.665992e+09,1.081198e+09,4.863265,4.892364,3.269346e+10
std,7.773743e+05,6.229386e+08,10.351497,10.384965,9.802637e+11
min,1.663071e+09,2.292617e+04,1.000000,1.000000,4.990000e+00
25%,1.665321e+09,5.231800e+08,1.000000,1.000000,6.031250e+01
50%,1.666041e+09,1.084561e+09,2.000000,2.000000,1.504500e+02
75%,1.666679e+09,1.624122e+09,5.000000,5.000000,3.342175e+02
max,1.667285e+09,2.147482e+09,408.000000,408.000000,3.430000e+13


More understandable information we can receive from describing the 'object' columns

In [54]:
df.describe(include='object')

,event_name,channel_grouping_user,device_category,item_category
count,711189,711189,711189,259699
unique,6,11,4,36
top,view_search_results,Paid Search,mobile,Electronics
freq,248194,212546,502196,56351


# Defining KPIs

### Getting distinct users number

In [51]:
df['user_pseudo_id'].nunique()

111544

### It is interesting how many unique customers purchused product from the store 

In [11]:
df.loc[df['event_name'] == 'purchase',['user_pseudo_id']].nunique()

user_pseudo_id    731
dtype: int64

### And how many of them purchased item more than once (iteracy dependent)

In [48]:
d = df.groupby('user_pseudo_id')['revenue'].count().sort_values(ascending=False)
d.head(235)

user_pseudo_id
1.402489e+08    21
5.117778e+08    16
7.295404e+07    14
8.426301e+08    10
1.719989e+09    10
                ..
1.894015e+09     2
1.298368e+09     1
1.896138e+09     1
1.398495e+09     1
1.023577e+09     1
Name: revenue, Length: 235, dtype: int64

### Let's see unique sessions number

In [55]:
df['session_id'].nunique()

194051

### Let's calculate the Conversion Rate. Let's take purchase as a "Desired action". When purchase is done, we receive revenue. It means, we have to divide 1234 (purchase actions) on 111547 (distinct users). 

In [65]:
df.loc[df['event_name'] == 'purchase',['revenue']].count() / df['user_pseudo_id'].nunique() * 100

revenue    1.10629
dtype: float64

### We can also take 'begin_checkout' and 'add_to_cart' records as KPIs to see and evaluate one the one hand how many people are not finishing the payment process and on the other hand how many users are not starting check out process at all. 

In [66]:
df.loc[df['event_name'] == 'begin_checkout'].count() / df['user_pseudo_id'].nunique() * 100

event_date               2.672488
event_name               2.672488
channel_grouping_user    2.672488
device_category          2.672488
session_id               2.672488
user_pseudo_id           2.672488
item_category            2.672488
unique_items             2.672488
item_quantity            2.672488
revenue                  0.000897
dtype: float64

In [67]:
df.loc[df['event_name'] == 'add_to_cart'].count() / df['user_pseudo_id'].nunique() * 100

event_date               23.943018
event_name               23.943018
channel_grouping_user    23.943018
device_category          23.943018
session_id               23.943018
user_pseudo_id           23.943018
item_category            23.943018
unique_items             23.943018
item_quantity            23.943018
revenue                   0.000000
dtype: float64

# Main user acquisition channel performance

In [68]:
df.groupby('channel_grouping_user')['event_name'].count().sort_values(ascending=False)

channel_grouping_user
Paid Search                  212546
Direct                       180745
Organic Search                98179
Organic Social                84318
Display                       69262
Unassigned                    31538
Paid Social                   16656
Email                         12467
Mobile Push Notifications      4812
Referral                        662
Organic Shopping                  4
Name: event_name, dtype: int64

In [69]:
a = df.groupby('channel_grouping_user')['revenue'].count().sort_values(ascending=False)
print(a)

channel_grouping_user
Direct                       569
Paid Search                  302
Organic Search               234
Organic Social                51
Email                         48
Display                       16
Unassigned                    10
Paid Social                    9
Referral                       1
Mobile Push Notifications      0
Organic Shopping               0
Name: revenue, dtype: int64


# Review data across the main device combinations and spot how they perform

In [70]:
df.groupby('device_category').count()

,event_date,event_name,channel_grouping_user,session_id,user_pseudo_id,item_category,unique_items,item_quantity,revenue
device_category,,,,,,,,,
desktop,175422,175422,175422,175422,175422,83691,83691,83691,480
mobile,502196,502196,502196,502196,502196,163030,163030,163030,716
smart tv,2678,2678,2678,2678,2678,774,774,774,0
tablet,30893,30893,30893,30893,30893,12204,12204,12204,44


In [71]:
b = df.groupby('device_category')['revenue'].count().sort_values(ascending=False)
print(b)

device_category
mobile      716
desktop     480
tablet       44
smart tv      0
Name: revenue, dtype: int64


# Exploring item category data

#### Which categories are manipulated (viewed, added to cart, starting to check out, purchased) mostly

In [72]:
df['item_category'].value_counts()

Electronics                          56351
Home & Kitchen                       34605
Clothing, Shoes & Jewelry            22639
Appliances                           20604
Sports & Outdoors                    17016
Cell Phones & Accessories            14196
Beauty & Personal Care               10649
Automotive                            9213
Tools & Home Improvement              7956
Patio, Lawn & Garden                  7918
Video Games                           7411
Industrial & Scientific               7261
Health & Household                    7147
Toys & Games                          7012
Baby Products                         6125
Arts, Crafts & Sewing                 5708
Office Products                       4946
Musical Instruments                   3479
Pet Supplies                          2637
Amazon Devices                        1351
Health, Household & Baby Care         1120
Handmade Products                     1104
Grocery & Gourmet Food                 769
Automotive 

#### Categories that are purchased mostly 

In [73]:
c = df.groupby('item_category')['revenue'].count().sort_values(ascending=False) 
print(c)

item_category
Electronics                          325
Home & Kitchen                       138
Clothing, Shoes & Jewelry             82
Appliances                            77
Beauty & Personal Care                71
Health & Household                    69
Sports & Outdoors                     66
Cell Phones & Accessories             65
Automotive                            65
Patio, Lawn & Garden                  40
Tools & Home Improvement              40
Industrial & Scientific               35
Office Products                       29
Baby Products                         25
Arts, Crafts & Sewing                 21
Video Games                           20
Toys & Games                          16
Pet Supplies                          12
Health, Household & Baby Care         10
Musical Instruments                    9
Garden & Outdoor                       6
Amazon Devices                         5
Automotive Parts & Accessories         4
Grocery & Gourmet Food                 3
Bo

#### Categories that bring the highest revenue

In [108]:
df.loc[df.event_name=='purchase'].groupby(['item_category']).agg({'revenue' : ['sum']})


,revenue
,sum
item_category,
Amazon Devices,2316.43000
Appliances,24723.86418
"Arts, Crafts & Sewing",10837.76206
Automotive,11512.23009
Automotive Parts & Accessories,1544.41000
Baby,29.72000
Baby Products,8434.81060
Beauty & Personal Care,21716.81000


#### Purchase to View Rate calculation
(PR = P/N where P is the number of purchases and N is the number of events during which a conversion could have occurred)

#### Add to Cart Rate calculation
(total number of sessions where someone adds an item to the cart and divide it by the total number of sessions)

In [95]:
df.loc[df['event_name'] == 'add_to_cart'].count() / df['session_id'].nunique() * 100

event_date               13.762877
event_name               13.762877
channel_grouping_user    13.762877
device_category          13.762877
session_id               13.762877
user_pseudo_id           13.762877
item_category            13.762877
unique_items             13.762877
item_quantity            13.762877
revenue                   0.000000
dtype: float64

#### Purchase to Cart Rate Calculation

In [100]:
df.loc[df['event_name'] == 'purchase'].count() / df.loc[df['event_name'] == 'add_to_cart'].count() * 100

event_date               4.620511
event_name               4.620511
channel_grouping_user    4.620511
device_category          4.620511
session_id               4.620511
user_pseudo_id           4.620511
item_category            4.620511
unique_items             4.620511
item_quantity            4.620511
revenue                       inf
dtype: float64

#### Calculating ASP

calculate avg revenues sorting buy items


In [112]:
df.loc[df.event_name=='purchase'].groupby(['item_category']).agg({'revenue' : ['mean']})


,revenue
,mean
item_category,
Amazon Devices,463.286000
Appliances,321.089145
"Arts, Crafts & Sewing",516.083908
Automotive,177.111232
Automotive Parts & Accessories,386.102500
Baby,29.720000
Baby Products,337.392424
Beauty & Personal Care,305.870563


### Now let's add a column with week day names and find out on which day do we have the most sales

In [115]:
df['day_of_week'] = df['event_date'].dt.day_name()
df.head()

,event_date,event_name,channel_grouping_user,device_category,session_id,user_pseudo_id,item_category,unique_items,item_quantity,revenue,day_of_week
0,2022-10-01,session_start,Paid Search,mobile,1664651196,1.029658e+09,NaN,NaN,NaN,NaN,Saturday
1,2022-10-01,session_start,Paid Search,mobile,1664656701,1.147645e+09,NaN,NaN,NaN,NaN,Saturday
2,2022-10-01,session_start,Paid Search,mobile,1664643763,1.483774e+07,NaN,NaN,NaN,NaN,Saturday
3,2022-10-01,view_search_results,Paid Search,mobile,1664643763,1.483774e+07,NaN,NaN,NaN,NaN,Saturday
4,2022-10-01,session_start,Paid Search,mobile,1664669276,1.618377e+09,NaN,NaN,NaN,NaN,Saturday


In [125]:
df.loc[df.event_name=='purchase'].groupby(['day_of_week']).agg({'revenue' : ['count']})

,revenue
,count
day_of_week,
Friday,157
Monday,174
Saturday,183
Sunday,156
Thursday,198
Tuesday,187
Wednesday,179


In [126]:
df.loc[df.event_name=='purchase'].groupby(['day_of_week']).agg({'revenue' : ['sum']})

,revenue
,sum
day_of_week,
Friday,48026.41476
Monday,62034.85351
Saturday,41643.80360
Sunday,91051.60681
Thursday,70262.85036
Tuesday,59643.46642
Wednesday,72240.83772
